In [ ]:
import openai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import argparse

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

def retrieve_documents(query, documents, index, k=3):
    """Retrieve the top-k most relevant documents for a given query."""
    query_embedding = model.encode([query], convert_to_numpy=True)
    _, indices = index.search(query_embedding, k)
    return [documents[i] for i in indices[0]]

def generate_response(prompt, retrieved_docs, api_key):
    """Generate an AI response using OpenAI GPT-4."""
    context = "\n".join(retrieved_docs)
    full_prompt = f"Use the following documents to answer the question:\n{context}\n\nQuestion: {prompt}\nAnswer:"
    
    openai.api_key = api_key
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": full_prompt}]
    )
    return response["choices"][0]["message"]["content"]

def check_sentence_grounding(response, retrieved_docs):
    """Check if each sentence in the response is grounded in retrieved documents."""
    response_sentences = response.split(". ")
    retrieved_embeddings = model.encode(retrieved_docs, convert_to_numpy=True)
    
    results = []
    for sentence in response_sentences:
        sentence_embedding = model.encode([sentence], convert_to_numpy=True)
        similarities = cosine_similarity(sentence_embedding, retrieved_embeddings)
        max_sim = max(similarities[0])
        is_grounded = max_sim > 0.7  # Threshold for similarity
        results.append((sentence, is_grounded, max_sim))
    
    return results

def compute_grounding_score(results):
    """Calculate the percentage of grounded sentences."""
    grounded_count = sum(1 for _, grounded, _ in results if grounded)
    total_sentences = len(results)
    return grounded_count / total_sentences if total_sentences > 0 else 0

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="RAG Groundedness Detection")
    parser.add_argument("--query", type=str, required=True, help="User question")
    parser.add_argument("--api_key", type=str, required=True, help="OpenAI API Key")
    args = parser.parse_args()
    
    # Sample corpus
    documents = [
        "Einstein developed the theory of relativity.",
        "The speed of light is approximately 299,792,458 meters per second.",
        "Black holes can warp spacetime."
    ]
    
    # Build FAISS index
    doc_embeddings = model.encode(documents, convert_to_numpy=True)
    index = faiss.IndexFlatL2(doc_embeddings.shape[1])
    index.add(np.array(doc_embeddings))
    
    # Retrieve documents & generate response
    retrieved_docs = retrieve_documents(args.query, documents, index)
    response = generate_response(args.query, retrieved_docs, args.api_key)
    
    # Check grounding & compute score
    grounding_results = check_sentence_grounding(response, retrieved_docs)
    grounding_score = compute_grounding_score(grounding_results)
    
    print("Generated Response:\n", response)
    print("\nGroundedness Score:", grounding_score)
    for sent, grounded, sim in grounding_results:
        print(f"Sentence: {sent}\nGrounded: {grounded} (Similarity: {sim:.2f})\n")
